## Librerias

In [1]:
import pandas as pd
import numpy as np
from flask import Flask, request 
from pymongo import MongoClient
import requests
import markdown.extensions.fenced_code

## Datos en Base de datos para Mongo

In [2]:
!ls

Data                                  config
Notebook Mongo_df & Calling_API.ipynb miapi.py
Notebook Sentiment_Analysis.ipynb     tools
README.md


In [3]:
data = pd.read_fwf("Data/SW_EpisodeIV.txt",header=None)
data

,0,1,2,3,4,5,6,7,8,9
0,"""character"" ""dialogue""",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"""1"" ""THREEPIO"" ""Did you hear that? They've sh...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"""2"" ""THREEPIO"" ""We're doomed!""",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"""3"" ""THREEPIO"" ""There'll be no escape for the ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"""4"" ""THREEPIO"" ""What's that?""",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
1006,"""1006"" ""LUKE"" ""Oh, no!""",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1007,"""1007"" ""THREEPIO"" ""Oh, my! Artoo! Can you he...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1008,"""1008"" ""TECHNICIAN"" ""We'll get to work on him ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1009,"""1009"" ""THREEPIO"" ""You must repair him! Sir, ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
del data[1]
del data[2]
del data[3]
del data[4]
del data[5]
del data[6]
del data[7]
del data[8]
del data[9]

In [5]:
data = data.iloc[1:]
data

,0
1,"""1"" ""THREEPIO"" ""Did you hear that? They've sh..."
2,"""2"" ""THREEPIO"" ""We're doomed!"""
3,"""3"" ""THREEPIO"" ""There'll be no escape for the ..."
4,"""4"" ""THREEPIO"" ""What's that?"""
5,"""5"" ""THREEPIO"" ""I should have known better tha..."
...,...
1006,"""1006"" ""LUKE"" ""Oh, no!"""
1007,"""1007"" ""THREEPIO"" ""Oh, my! Artoo! Can you he..."
1008,"""1008"" ""TECHNICIAN"" ""We'll get to work on him ..."
1009,"""1009"" ""THREEPIO"" ""You must repair him! Sir, ..."


In [6]:
sw_data = data[0].str.split(" ", n = 2, expand = True)
sw_data.rename(columns={0:'Line_order', 1:'Character', 2:'Dialogue'}, inplace=True)
sw_data.Character = sw_data.Character.apply(lambda x: x.strip('"'))
sw_data.Line_order = sw_data.Line_order.apply(lambda x: x.strip('"'))
sw_data.Dialogue = sw_data.Dialogue.apply(lambda x: x.strip('"'))

In [7]:
sw_data

,Line_order,Character,Dialogue
1,1,THREEPIO,Did you hear that? They've shut down the main...
2,2,THREEPIO,We're doomed!
3,3,THREEPIO,There'll be no escape for the Princess this time.
4,4,THREEPIO,What's that?
5,5,THREEPIO,I should have known better than to trust the l...
...,...,...,...
1006,1006,LUKE,"Oh, no!"
1007,1007,THREEPIO,"Oh, my! Artoo! Can you hear me? Say somethi..."
1008,1008,TECHNICIAN,We'll get to work on him right away.
1009,1009,THREEPIO,"You must repair him! Sir, if any of my circui..."


Transform to Json

In [8]:
sw_data.to_json("Data/SW_EpisodeIV.json", orient="records")

## Llamando API

Getting Dialogue and Characters from Line_order:

In [43]:
url_query_line_order = "http://localhost:5000/line_order/"
line = "800"
line_order = requests.get(url_query_line_order + line)
line_order.json()

[{'Line_order': '800',
  'Character': 'LEIA',
  'Dialogue': "We don't have time for our sorrows, commander.  The battle station has surely tracked us here.It's the only explanation for the ease of our escape.  You must use the information in this R2 unit to plan the attack.  It is our only hope."}]

Getting Dialogue from Character:

In [32]:
url_query_dialogue = "http://localhost:5000/dialogue/"
person = "VADER"
dialogue = requests.get(url_query_dialogue + person)
dialogue.json()

[{'Line_order': '12',
  'Character': 'VADER',
  'Dialogue': 'Where are those transmissions you intercepted?'},
 {'Line_order': '14',
  'Character': 'VADER',
  'Dialogue': 'If this is a consular ship... where is the Ambassador?'},
 {'Line_order': '15',
  'Character': 'VADER',
  'Dialogue': "Commander, tear this ship apart until you've found those plans and bring me the Ambassador.  I want her alive!"},
 {'Line_order': '43',
  'Character': 'VADER',
  'Dialogue': "Don't play games with me, Your Highness.  You weren't on any mercy mission this time.  You passed directly through a restricted system.  Several transmissions were beamed to this ship by Rebel spies.  I want to know what happened to the plans they sent you."},
 {'Line_order': '45',
  'Character': 'VADER',
  'Dialogue': "You're a part of the Rebel Alliance... and a traitor.  Take her away!"},
 {'Line_order': '49',
  'Character': 'VADER',
  'Dialogue': 'Leave that to me.  Send a distress signal and then inform the senate that all 

Polarity:

In [57]:
url_query_pol = "http://localhost:5000/polarity/"
new = requests.get(url_query_pol)

Compound:

In [56]:
url_query_com = "http://localhost:5000/compound/"
new = requests.get(url_query_com)

Posting new dialogue for character in specific line:

In [51]:
url_query_newpost = "http://localhost:5000/new_dialogue/"

In [52]:
datos = {
"Line_order": "944",
"Character": "VADER",
"Dialogue": "Dont let that m***** f***** escape"
}

In [53]:
new_post = requests.post(url_query_newpost, data = datos)
new_post

<Response [404]>